In [5]:
import numpy as np
import pandas as pd

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

In [7]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [9]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
df = pd.read_excel('/content/final health datast.xlsx')
display(df.head())
display(df.info())

,user_id,date,Age,Gender,Blood Type,Occupation,BMI Category,Medical Condition,steps per day,calories_burned,...,Doctor,Hospital,Admission Type,Model,Category,Heart_Rate_Accuracy_Percent,Step_Count_Accuracy_Percent,Sleep_Tracking_Accuracy_Percent.1,User_Satisfaction_Rating,Battery_Life_Hours
0,468,2023-01-01,30.0,Male,B-,Software Engineer,Overweight,Cancer,4530.0,2543.02,...,Matthew Smith,Sons and Miller,Urgent,Inspire 4,Fitness Tracker,89.69,93.03,78.91,6.5,129.9
1,879,2023-01-01,62.0,Male,A+,Doctor,Normal,Obesity,11613.0,1720.76,...,Samantha Davies,Kim Inc,Emergency,Watch SE 3,Smartwatch,95.92,98.20,79.76,8.3,26.5
2,152,2023-01-01,76.0,Female,A-,Doctor,Normal,Obesity,27335.0,1706.35,...,Tiffany Mitchell,Cook PLC,Emergency,Versa 4,Sports Watch,92.24,96.81,74.49,6.0,161.2
3,311,2023-01-01,28.0,Female,O+,Sales Representative,Obese,Diabetes,13459.0,2912.38,...,Kevin Wells,"Hernandez Rogers and Vang,",Elective,Vantage V3,Smartwatch,96.77,95.56,78.06,8.0,69.4
4,759,2023-01-01,43.0,Female,AB+,Sales Representative,Obese,Cancer,15378.0,3344.51,...,Kathleen Hanna,White-White,Urgent,Galaxy Watch FE,Smartwatch,92.27,98.15,75.23,8.3,39.7


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929771 entries, 0 to 929770
Data columns (total 36 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   user_id                            133885 non-null  object        
 1   date                               374 non-null     datetime64[ns]
 2   Age                                374 non-null     float64       
 3   Gender                             374 non-null     object        
 4   Blood Type                         374 non-null     object        
 5   Occupation                         374 non-null     object        
 6   BMI Category                       374 non-null     object        
 7   Medical Condition                  374 non-null     object        
 8   steps per day                      374 non-null     float64       
 9   calories_burned                    374 non-null     float64       
 10  distance_km         

None

In [12]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
print("Columns with missing values and their counts:")
print(missing_values)

Columns with missing values and their counts:
workout_type                         929443
date                                 929397
Gender                               929397
Age                                  929397
Stress Level                         929397
Blood Type                           929397
BMI Category                         929397
Occupation                           929397
steps per day                        929397
calories_burned                      929397
distance_km                          929397
Medical Condition                    929397
active_minutes                       929397
Physical Activity Level              929397
Heart Rate                           929397
heart_rate_avg                       929397
Sleep Duration                       929397
Quality of Sleep                     929397
Sleep_Tracking_Accuracy_Percent      929397
Blood Pressure                       929397
Heart_Rate_Accuracy_Percent          929397
Test Results                  

In [13]:
cols_to_drop = missing_values[missing_values > 929000].index.tolist()
cols_to_drop.append('user_id')
df_cleaned = df.drop(columns=cols_to_drop)
print("Columns dropped due to high missing values or irrelevance:")
print(cols_to_drop)
print("\nRemaining columns:")
print(df_cleaned.columns.tolist())

Columns dropped due to high missing values or irrelevance:
['workout_type', 'date', 'Gender', 'Age', 'Stress Level', 'Blood Type', 'BMI Category', 'Occupation', 'steps per day', 'calories_burned', 'distance_km', 'Medical Condition', 'active_minutes', 'Physical Activity Level', 'Heart Rate', 'heart_rate_avg', 'Sleep Duration', 'Quality of Sleep', 'Sleep_Tracking_Accuracy_Percent', 'Blood Pressure', 'Heart_Rate_Accuracy_Percent', 'Test Results', 'Doctor', 'Medication', 'Hospital', 'Category', 'Model', 'Admission Type', 'mood', 'location', 'weather_conditions', 'user_id']

Remaining columns:
['Step_Count_Accuracy_Percent', 'Sleep_Tracking_Accuracy_Percent.1', 'User_Satisfaction_Rating', 'Battery_Life_Hours']


In [14]:
for col in df_cleaned.columns:
    if df_cleaned[col].isnull().sum() > 0:
        if df_cleaned[col].dtype in ['float64', 'int64']:
            df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].mean())
        else:
            df_cleaned
            df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].mode()[0])

print("\nMissing values after imputation:")
print(df_cleaned.isnull().sum())


Missing values after imputation:
Step_Count_Accuracy_Percent          0
Sleep_Tracking_Accuracy_Percent.1    0
User_Satisfaction_Rating             0
Battery_Life_Hours                   0
dtype: int64


In [15]:
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_cleaned), columns=df_cleaned.columns)
display(df_scaled.head())

,Step_Count_Accuracy_Percent,Sleep_Tracking_Accuracy_Percent.1,User_Satisfaction_Rating,Battery_Life_Hours
0,-34.058892,0.418063,-8.785648,-2.585671
1,27.144990,3.744429,1.512292,-11.300139
2,10.689788,-16.879036,-11.646187,0.052268
3,-4.108056,-2.908302,-0.204032,-7.684562
4,26.553076,-13.983142,1.512292,-10.187653


In [16]:
y = df_cleaned['User_Satisfaction_Rating']
print("Target variable selected: 'User_Satisfaction_Rating'")

Target variable selected: 'User_Satisfaction_Rating'


***Feature Engineering***

In [26]:

physiological_features = ['Heart Rate', 'Sleep Duration', 'Stress Level'] # Assuming these column names exist
lifestyle_features = ['steps per day', 'calories_burned', 'active_minutes'] # Assuming these column names exist
medical_features = ['Age', 'BMI Category', 'Medical Condition'] # Assuming these column names exist

selected_features = physiological_features + lifestyle_features + medical_features

if 'date' in df.columns:
  selected_features.append('date')

df_features = df[selected_features].copy()

display(df_features.head())
display(df_features.info())

,Heart Rate,Sleep Duration,Stress Level,steps per day,calories_burned,active_minutes,Age,BMI Category,Medical Condition,date
0,77.0,6.1,6.0,4530.0,2543.02,613.0,30.0,Overweight,Cancer,2023-01-01
1,75.0,6.2,8.0,11613.0,1720.76,352.0,62.0,Normal,Obesity,2023-01-01
2,75.0,6.2,8.0,27335.0,1706.35,236.0,76.0,Normal,Obesity,2023-01-01
3,85.0,5.9,8.0,13459.0,2912.38,1329.0,28.0,Obese,Diabetes,2023-01-01
4,85.0,5.9,8.0,15378.0,3344.51,52.0,43.0,Obese,Cancer,2023-01-01


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929771 entries, 0 to 929770
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Heart Rate         374 non-null    float64       
 1   Sleep Duration     374 non-null    float64       
 2   Stress Level       374 non-null    float64       
 3   steps per day      374 non-null    float64       
 4   calories_burned    374 non-null    float64       
 5   active_minutes     374 non-null    float64       
 6   Age                374 non-null    float64       
 7   BMI Category       374 non-null    object        
 8   Medical Condition  374 non-null    object        
 9   date               374 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 70.9+ MB


None

In [27]:
# Check for missing values in the selected features DataFrame
missing_values_features = df_features.isnull().sum()
missing_values_features = missing_values_features[missing_values_features > 0].sort_values(ascending=False)
print("Missing values in selected features before imputation:")
print(missing_values_features)

# Impute missing values in df_features
for col in df_features.columns:
    if df_features[col].isnull().sum() > 0:
        if df_features[col].dtype in ['float64', 'int64']:
            # Impute numerical columns with the mean
            df_features[col] = df_features[col].fillna(df_features[col].mean())
        else:
            # Impute categorical columns with the mode
            df_features[col] = df_features[col].fillna(df_features[col].mode()[0])

print("\nMissing values in selected features after imputation:")
print(df_features.isnull().sum())

Missing values in selected features before imputation:
Heart Rate           929397
Sleep Duration       929397
Stress Level         929397
steps per day        929397
calories_burned      929397
active_minutes       929397
Age                  929397
BMI Category         929397
Medical Condition    929397
date                 929397
dtype: int64

Missing values in selected features after imputation:
Heart Rate           0
Sleep Duration       0
Stress Level         0
steps per day        0
calories_burned      0
active_minutes       0
Age                  0
BMI Category         0
Medical Condition    0
date                 0
dtype: int64


In [28]:
# Encode categorical features
df_features_encoded = pd.get_dummies(df_features, columns=['BMI Category', 'Medical Condition'], drop_first=True)

display(df_features_encoded.head())
display(df_features_encoded.info())

,Heart Rate,Sleep Duration,Stress Level,steps per day,calories_burned,active_minutes,Age,date,BMI Category_Normal Weight,BMI Category_Obese,BMI Category_Overweight,Medical Condition_Asthma,Medical Condition_Cancer,Medical Condition_Diabetes,Medical Condition_Hypertension,Medical Condition_Obesity
0,77.0,6.1,6.0,4530.0,2543.02,613.0,30.0,2023-01-01,False,False,True,False,True,False,False,False
1,75.0,6.2,8.0,11613.0,1720.76,352.0,62.0,2023-01-01,False,False,False,False,False,False,False,True
2,75.0,6.2,8.0,27335.0,1706.35,236.0,76.0,2023-01-01,False,False,False,False,False,False,False,True
3,85.0,5.9,8.0,13459.0,2912.38,1329.0,28.0,2023-01-01,False,True,False,False,False,True,False,False
4,85.0,5.9,8.0,15378.0,3344.51,52.0,43.0,2023-01-01,False,True,False,False,True,False,False,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929771 entries, 0 to 929770
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Heart Rate                      929771 non-null  float64       
 1   Sleep Duration                  929771 non-null  float64       
 2   Stress Level                    929771 non-null  float64       
 3   steps per day                   929771 non-null  float64       
 4   calories_burned                 929771 non-null  float64       
 5   active_minutes                  929771 non-null  float64       
 6   Age                             929771 non-null  float64       
 7   date                            929771 non-null  datetime64[ns]
 8   BMI Category_Normal Weight      929771 non-null  bool          
 9   BMI Category_Obese              929771 non-null  bool          
 10  BMI Category_Overweight         929771 non-null  bool   

None

In [29]:
# Select numerical columns for scaling
numerical_cols = df_features_encoded.select_dtypes(include=np.number).columns.tolist()
if 'date' in numerical_cols:
  numerical_cols.remove('date')

# Apply StandardScaler to the numerical columns
scaler = StandardScaler()
df_features_encoded[numerical_cols] = scaler.fit_transform(df_features_encoded[numerical_cols])

display(df_features_encoded.head())
display(df_features_encoded.info())

,Heart Rate,Sleep Duration,Stress Level,steps per day,calories_burned,active_minutes,Age,date,BMI Category_Normal Weight,BMI Category_Obese,BMI Category_Overweight,Medical Condition_Asthma,Medical Condition_Cancer,Medical Condition_Diabetes,Medical Condition_Hypertension,Medical Condition_Obesity
0,82.504259,-64.762491,17.302440,-58.990311,-12.652974,-9.650441,-48.378128,2023-01-01,False,False,True,False,True,False,False,False
1,58.359820,-58.487576,73.572982,-17.967353,-68.388499,-41.626618,28.779313,2023-01-01,False,False,False,False,False,False,False,True
2,58.359820,-58.487576,73.572982,73.090525,-69.365257,-55.838253,62.535693,2023-01-01,False,False,False,False,False,False,False,True
3,179.082016,-77.312321,73.572982,-7.275784,12.383479,78.069646,-53.200468,2023-01-01,False,True,False,False,False,True,False,False
4,179.082016,-77.312321,73.572982,3.838582,41.674692,-78.380845,-17.032918,2023-01-01,False,True,False,False,True,False,False,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929771 entries, 0 to 929770
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Heart Rate                      929771 non-null  float64       
 1   Sleep Duration                  929771 non-null  float64       
 2   Stress Level                    929771 non-null  float64       
 3   steps per day                   929771 non-null  float64       
 4   calories_burned                 929771 non-null  float64       
 5   active_minutes                  929771 non-null  float64       
 6   Age                             929771 non-null  float64       
 7   date                            929771 non-null  datetime64[ns]
 8   BMI Category_Normal Weight      929771 non-null  bool          
 9   BMI Category_Obese              929771 non-null  bool          
 10  BMI Category_Overweight         929771 non-null  bool   

None

In [30]:
print(y.dtype)

object


## Define features



In [31]:
X = df_scaled.drop(columns=['User_Satisfaction_Rating'])
display(X.head())
display(X.info())

,Step_Count_Accuracy_Percent,Sleep_Tracking_Accuracy_Percent.1,Battery_Life_Hours
0,-34.058892,0.418063,-2.585671
1,27.144990,3.744429,-11.300139
2,10.689788,-16.879036,0.052268
3,-4.108056,-2.908302,-7.684562
4,26.553076,-13.983142,-10.187653


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929771 entries, 0 to 929770
Data columns (total 3 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Step_Count_Accuracy_Percent        929771 non-null  float64
 1   Sleep_Tracking_Accuracy_Percent.1  929771 non-null  float64
 2   Battery_Life_Hours                 929771 non-null  float64
dtypes: float64(3)
memory usage: 21.3 MB


None

## Split data


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (743816, 3)
Shape of X_test: (185955, 3)
Shape of y_train: (743816,)
Shape of y_test: (185955,)


***Model Evaluation***

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Choose 'mood' as the target variable
y = df['mood']

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode the target variable
y_encoded = label_encoder.fit_transform(y)

print("Original target variable values:", y.unique())
print("Encoded target variable values:", y_encoded)

Original target variable values: ['Tired' 'Happy' 'Neutral' 'Stressed' nan]
Encoded target variable values: [3 0 1 ... 4 4 4]


In [ ]:
# Drop the 'date' column from the features DataFrame
X = df_features_encoded.drop(columns=['date'])

# Define the target variable (encoded 'mood')
# y is already defined and encoded in the previous step as y_encoded

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (743816, 15)
Shape of X_test: (185955, 15)
Shape of y_train: (743816,)
Shape of y_test: (185955,)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_logistic = logistic_model.predict(X_test)

# Evaluate the model
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
report_logistic = classification_report(y_test, y_pred_logistic)

print("Logistic Regression Model Performance:")
print(f"Accuracy: {accuracy_logistic:.4f}")
print("Classification Report:")
print(report_logistic)

Logistic Regression Model Performance:
Accuracy: 0.9995
Classification Report:
              precision    recall  f1-score   support

           0       0.23      0.10      0.14        29
           1       0.20      0.04      0.07        23
           2       0.22      0.19      0.20        27
           3       0.17      0.09      0.12        33
           4       1.00      1.00      1.00    185843

    accuracy                           1.00    185955
   macro avg       0.36      0.28      0.31    185955
weighted avg       1.00      1.00      1.00    185955



In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the Random Forest model
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
random_forest_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_random_forest = random_forest_model.predict(X_test)

# Evaluate the model
accuracy_random_forest = accuracy_score(y_test, y_pred_random_forest)
report_random_forest = classification_report(y_test, y_pred_random_forest)

print("Random Forest Classifier Model Performance:")
print(f"Accuracy: {accuracy_random_forest:.4f}")
print("Classification Report:")
print(report_random_forest)

Random Forest Classifier Model Performance:
Accuracy: 0.9995
Classification Report:
              precision    recall  f1-score   support

           0       0.30      0.21      0.24        29
           1       0.22      0.22      0.22        23
           2       0.32      0.26      0.29        27
           3       0.20      0.06      0.09        33
           4       1.00      1.00      1.00    185843

    accuracy                           1.00    185955
   macro avg       0.41      0.35      0.37    185955
weighted avg       1.00      1.00      1.00    185955



In [ ]:
from xgboost import XGBClassifier

# Initialize and train the XGBoost model
# Use 'objective' and 'eval_metric' for multi-class classification
# and 'use_label_encoder=False' to avoid a deprecation warning
# and set 'n_estimators' to a reasonable value
xgb_model = XGBClassifier(objective='multi:softmax', eval_metric='mlogloss', use_label_encoder=False, n_estimators=100, random_state=42, n_jobs=-1)
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate the model
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
report_xgb = classification_report(y_test, y_pred_xgb)

print("XGBoost Classifier Model Performance:")
print(f"Accuracy: {accuracy_xgb:.4f}")
print("Classification Report:")
print(report_xgb)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [06:31:01] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost Classifier Model Performance:
Accuracy: 0.9995
Classification Report:
              precision    recall  f1-score   support

           0       0.36      0.28      0.31        29
           1       0.25      0.22      0.23        23
           2       0.22      0.19      0.20        27
           3       0.30      0.09      0.14        33
           4       1.00      1.00      1.00    185843

    accuracy                           1.00    185955
   macro avg       0.43      0.35      0.38    185955
weighted avg       1.00      1.00      1.00    185955



In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, classification_report

# Get the number of classes from the unique values in the encoded target variable
num_classes = len(np.unique(y_encoded))

# Build the Neural Network model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax') # Use softmax for multi-class classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', # Use sparse_categorical_crossentropy for integer encoded labels
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy_nn = model.evaluate(X_test, y_test)

# Make predictions
y_pred_nn = np.argmax(model.predict(X_test), axis=-1)

# Print performance metrics
print("\nNeural Network Model Performance:")
print(f"Accuracy: {accuracy_nn:.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred_nn))

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


18596/18596 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - accuracy: 0.9987 - loss: 0.0385 - val_accuracy: 0.9994 - val_loss: 0.0043
Epoch 2/10
18596/18596 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.9994 - loss: 0.0049 - val_accuracy: 0.9994 - val_loss: 0.0037
Epoch 3/10
18596/18596 ━━━━━━━━━━━━━━━━━━━━ 43s 2ms/step - accuracy: 0.9994 - loss: 0.0060 - val_accuracy: 0.9994 - val_loss: 0.0038
Epoch 4/10
18596/18596 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.9994 - loss: 0.0053 - val_accuracy: 0.9994 - val_loss: 0.0039
Epoch 5/10
18596/18596 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - accuracy: 0.9994 - loss: 0.0045 - val_accuracy: 0.9994 - val_loss: 0.0116
Epoch 6/10
18596/18596 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - accuracy: 0.9994 - loss: 0.0080 - val_accuracy: 0.9994 - val_loss: 0.0048
Epoch 7/10
18596/18596 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - accuracy: 0.9995 - loss: 0.0142 - val_accuracy: 0.9994 - val_loss: 0.0035
Epoch 8/10
18596/18596 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.9995 - lo

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


***HyperParameter Tuning***

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter distribution for Random Forest
# Using a slightly reduced parameter grid for efficiency
param_grid_rf = {
    'n_estimators': [100, 200], # Reduced options
    'max_depth': [None, 10, 20], # Reduced options
    'min_samples_split': [2, 5], # Reduced options
    'min_samples_leaf': [1, 2], # Reduced options
    'class_weight': [None, 'balanced']
}

# Initialize the RandomizedSearchCV object
# Use f1_weighted as the scoring metric to account for class imbalance
# Reduced n_iter for faster execution
random_search_rf = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=42, n_jobs=-1),
                                      param_distributions=param_grid_rf,
                                      n_iter=50, # Reduced number of iterations
                                      scoring='f1_weighted',
                                      cv=3, # Using 3-fold cross-validation
                                      verbose=2,
                                      random_state=42,
                                      n_jobs=-1)

# Fit the RandomizedSearchCV to the training data
random_search_rf.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters for Random Forest (Randomized Search):", random_search_rf.best_params_)
print("Best F1-weighted score for Random Forest (Randomized Search):", random_search_rf.best_score_)

# Get the best model
best_random_forest_model = random_search_rf.best_estimator_

# Evaluate the best model on the test set
y_pred_best_random_forest = best_random_forest_model.predict(X_test)

# Print the classification report for the best model
print("\nClassification Report for Best Random Forest Model (Randomized Search):")
print(classification_report(y_test, y_pred_best_random_forest))

Fitting 3 folds for each of 48 candidates, totalling 144 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 48 is smaller than n_iter=50. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for Random Forest (Randomized Search): {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 10, 'class_weight': 'balanced'}
Best F1-weighted score for Random Forest (Randomized Search): 0.9993708892849309

Classification Report for Best Random Forest Model (Randomized Search):
              precision    recall  f1-score   support

           0       0.33      0.14      0.20        29
           1       0.23      0.22      0.22        23
           2       0.27      0.26      0.26        27
           3       0.40      0.18      0.25        33
           4       1.00      1.00      1.00    185843

    accuracy                           1.00    185955
   macro avg       0.45      0.36      0.39    185955
weighted avg       1.00      1.00      1.00    185955



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

# Define the parameter distribution for XGBoost
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.9, 1.0],
    'lambda': [1, 1.5, 2], # L2 regularization
    'alpha': [0, 0.5, 1], # L1 regularization
    'scale_pos_weight': [1, (y_train == 0).sum() / (y_train == 1).sum()] # Example for binary, need adjustment for multiclass
}

# Note: For multiclass classification with imbalanced data,
# 'scale_pos_weight' is typically not used directly.
# Instead, consider using techniques like oversampling or undersampling
# or setting the 'objective' to include class weights if supported by XGBoost.
# For now, I will remove scale_pos_weight for simplicity in this multiclass case
# and recommend addressing imbalance with data-level techniques if needed.

param_grid_xgb = {
    'n_estimators': [100, 200], # Reduced options
    'max_depth': [3, 5, 7], # Reduced options
    'learning_rate': [0.01, 0.1], # Reduced options
    'subsample': [0.8, 1.0], # Reduced options
    'colsample_bytree': [0.8, 1.0], # Reduced options
    'lambda': [1, 1.5], # Reduced options
    'alpha': [0, 0.5] # Reduced options
}

# Use f1_weighted as the scoring metric for multi-class imbalance
random_search_xgb = RandomizedSearchCV(estimator=XGBClassifier(objective='multi:softmax', eval_metric='mlogloss', use_label_encoder=False, random_state=42, n_jobs=-1),
                                       param_distributions=param_grid_xgb,
                                       n_iter=50, # Number of iterations (adjust as needed)
                                       scoring='f1_weighted',
                                       cv=3, # Using 3-fold cross-validation
                                       verbose=2,
                                       random_state=42,
                                       n_jobs=-1)

# Fit the RandomizedSearchCV to the training data
random_search_xgb.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters for XGBoost (Randomized Search):", random_search_xgb.best_params_)
print("Best F1-weighted score for XGBoost (Randomized Search):", random_search_xgb.best_score_)

# Get the best model
best_xgb_model = random_search_xgb.best_estimator_

# Evaluate the best model on the test set
y_pred_best_xgb = best_xgb_model.predict(X_test)

# Print the classification report for the best model
print("\nClassification Report for Best XGBoost Model (Randomized Search):")
print(classification_report(y_test, y_pred_best_xgb))

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [07:35:03] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best parameters for XGBoost (Randomized Search): {'subsample': 1.0, 'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.01, 'lambda': 1.5, 'colsample_bytree': 1.0, 'alpha': 0.5}
Best F1-weighted score for XGBoost (Randomized Search): 0.9993845412940376

Classification Report for Best XGBoost Model (Randomized Search):
              precision    recall  f1-score   support

           0       0.29      0.21      0.24        29
           1       0.26      0.22      0.24        23
           2       0.24      0.19      0.21        27
           3       0.43      0.18      0.26        33
           4       1.00      1.00      1.00    185843

    accuracy                           1.00    185955
   macro avg       0.44      0.36      0.39    185955
weighted avg       1.00      1.00      1.00    185955



***'Medical Condition'*** as the target variable (y).


In [21]:
y = df['Medical Condition']
display(y.head())
print(y.dtype)

,Medical Condition
0,Cancer
1,Obesity
2,Obesity
3,Diabetes
4,Cancer


object


In [22]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("Original target variable unique values:", y.unique())
print("Encoded target variable unique values:", np.unique(y_encoded))

Original target variable unique values: ['Cancer' 'Obesity' 'Diabetes' 'Asthma' 'Hypertension' 'Arthritis' nan]
Encoded target variable unique values: [0 1 2 3 4 5 6]


In [33]:
# Create the feature DataFrame by dropping the 'Medical Condition' related columns
X = df_features_encoded.drop(columns=['Medical Condition_Cancer', 'Medical Condition_Diabetes',
                                      'Medical Condition_Asthma', 'Medical Condition_Hypertension',
                                      'Medical Condition_Obesity'])

display(X.head())
display(X.info())

,Heart Rate,Sleep Duration,Stress Level,steps per day,calories_burned,active_minutes,Age,date,BMI Category_Normal Weight,BMI Category_Obese,BMI Category_Overweight
0,82.504259,-64.762491,17.302440,-58.990311,-12.652974,-9.650441,-48.378128,2023-01-01,False,False,True
1,58.359820,-58.487576,73.572982,-17.967353,-68.388499,-41.626618,28.779313,2023-01-01,False,False,False
2,58.359820,-58.487576,73.572982,73.090525,-69.365257,-55.838253,62.535693,2023-01-01,False,False,False
3,179.082016,-77.312321,73.572982,-7.275784,12.383479,78.069646,-53.200468,2023-01-01,False,True,False
4,179.082016,-77.312321,73.572982,3.838582,41.674692,-78.380845,-17.032918,2023-01-01,False,True,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929771 entries, 0 to 929770
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Heart Rate                  929771 non-null  float64       
 1   Sleep Duration              929771 non-null  float64       
 2   Stress Level                929771 non-null  float64       
 3   steps per day               929771 non-null  float64       
 4   calories_burned             929771 non-null  float64       
 5   active_minutes              929771 non-null  float64       
 6   Age                         929771 non-null  float64       
 7   date                        929771 non-null  datetime64[ns]
 8   BMI Category_Normal Weight  929771 non-null  bool          
 9   BMI Category_Obese          929771 non-null  bool          
 10  BMI Category_Overweight     929771 non-null  bool          
dtypes: bool(3), datetime64[ns](1), float64(

None

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (743816, 11)
Shape of X_test: (185955, 11)
Shape of y_train: (743816,)
Shape of y_test: (185955,)


In [35]:
print("Class distribution in y_train before handling imbalance:")
print(pd.Series(y_train).value_counts())

Class distribution in y_train before handling imbalance:
6    743517
1        56
3        53
0        50
4        49
2        48
5        43
Name: count, dtype: int64


In [36]:
X_train = X_train.drop(columns=['date'])
X_test = X_test.drop(columns=['date'])

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("\nClass distribution in y_train after SMOTE:")
print(pd.Series(y_train_resampled).value_counts())


Class distribution in y_train after SMOTE:
6    743517
2    743517
1    743517
4    743517
0    743517
5    743517
3    743517
Name: count, dtype: int64


In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, classification_report

# 1. Initialize and train the Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train_resampled, y_train_resampled)
print("Logistic Regression model trained.")

# 3. Initialize and train the Random Forest model
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
random_forest_model.fit(X_train_resampled, y_train_resampled)
print("Random Forest model trained.")

# 5. Initialize and train the XGBoost model
xgb_model = XGBClassifier(objective='multi:softmax', eval_metric='mlogloss', use_label_encoder=False, n_estimators=100, random_state=42, n_jobs=-1)
xgb_model.fit(X_train_resampled, y_train_resampled)
print("XGBoost model trained.")

# 7. Get the number of unique classes
num_classes = len(np.unique(y_train_resampled))
print(f"Number of classes: {num_classes}")

# 8. Build the Neural Network model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_resampled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])
print("Neural Network model built.")

# 9. Compile the Neural Network model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print("Neural Network model compiled.")

# 10. Train the Neural Network model
history = model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, validation_split=0.2)
print("Neural Network model trained.")

# 11. Evaluate the Neural Network model on the test set
loss, accuracy_nn = model.evaluate(X_test, y_test)

# 12. Make predictions
y_pred_nn = np.argmax(model.predict(X_test), axis=-1)

# 13. Print performance metrics
print("\nNeural Network Model Performance:")
print(f"Accuracy: {accuracy_nn:.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred_nn))

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression model trained.
Random Forest model trained.


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [13:04:38] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost model trained.
Number of classes: 7


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Neural Network model built.
Neural Network model compiled.
Epoch 1/10
130116/130116 ━━━━━━━━━━━━━━━━━━━━ 342s 3ms/step - accuracy: 0.7875 - loss: 0.5696 - val_accuracy: 0.2736 - val_loss: 65.5118
Epoch 2/10
130116/130116 ━━━━━━━━━━━━━━━━━━━━ 433s 3ms/step - accuracy: 0.8949 - loss: 0.2829 - val_accuracy: 0.2764 - val_loss: 77.1474
Epoch 3/10
130116/130116 ━━━━━━━━━━━━━━━━━━━━ 418s 3ms/step - accuracy: 0.9098 - loss: 0.2508 - val_accuracy: 0.2812 - val_loss: 88.6758
Epoch 4/10
130116/130116 ━━━━━━━━━━━━━━━━━━━━ 402s 3ms/step - accuracy: 0.9170 - loss: 0.2354 - val_accuracy: 0.2797 - val_loss: 87.2432
Epoch 5/10
130116/130116 ━━━━━━━━━━━━━━━━━━━━ 387s 3ms/step - accuracy: 0.9228 - loss: 0.2233 - val_accuracy: 0.2813 - val_loss: 102.3499
Epoch 6/10
130116/130116 ━━━━━━━━━━━━━━━━━━━━ 380s 3ms/step - accuracy: 0.9265 - loss: 0.2141 - val_accuracy: 0.2818 - val_loss: 104.1879
Epoch 7/10
130116/130116 ━━━━━━━━━━━━━━━━━━━━ 356s 3ms/step - accuracy: 0.9294 - loss: 0.2092 - val_accuracy: 0.2816 

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the Random Forest model using the resampled data
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
random_forest_model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set (using the original X_test)
y_pred_random_forest = random_forest_model.predict(X_test)

# Evaluate the model
accuracy_random_forest = accuracy_score(y_test, y_pred_random_forest)
report_random_forest = classification_report(y_test, y_pred_random_forest)

print("Random Forest Classifier Model Performance (with SMOTE):")
print(f"Accuracy: {accuracy_random_forest:.4f}")
print("Classification Report:")
print(report_random_forest)

Random Forest Classifier Model Performance (with SMOTE):
Accuracy: 0.9997
Classification Report:
              precision    recall  f1-score   support

           0       0.25      0.60      0.35        10
           1       0.08      0.09      0.08        11
           2       0.21      0.21      0.21        14
           3       0.10      0.07      0.08        15
           4       0.09      0.09      0.09        11
           5       0.00      0.00      0.00        14
           6       1.00      1.00      1.00    185880

    accuracy                           1.00    185955
   macro avg       0.25      0.29      0.26    185955
weighted avg       1.00      1.00      1.00    185955



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the Logistic Regression model using the resampled data
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set (using the original X_test)
y_pred_logistic = logistic_model.predict(X_test)

# Evaluate the model
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
report_logistic = classification_report(y_test, y_pred_logistic)

print("Logistic Regression Model Performance (with SMOTE):")
print(f"Accuracy: {accuracy_logistic:.4f}")
print("Classification Report:")
print(report_logistic)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Model Performance (with SMOTE):
Accuracy: 0.9997
Classification Report:
              precision    recall  f1-score   support

           0       0.25      0.20      0.22        10
           1       0.20      0.18      0.19        11
           2       0.20      0.07      0.11        14
           3       0.14      0.07      0.09        15
           4       0.10      0.09      0.10        11
           5       0.21      0.21      0.21        14
           6       1.00      1.00      1.00    185880

    accuracy                           1.00    185955
   macro avg       0.30      0.26      0.27    185955
weighted avg       1.00      1.00      1.00    185955



In [39]:
import joblib
joblib.dump(model, "smartwatch_ai.pkl")



['smartwatch_ai.pkl']

In [40]:
from fastapi import FastAPI
import joblib
import numpy as np

app = FastAPI()
model = joblib.load("smartwatch_ai.pkl")

@app.post("/predict")
def predict(data: dict):
    features = np.array([data['heart_rate'], data['spo2'], data['sleep'], data['stress']]).reshape(1, -1)
    prediction = model.predict(features)
    return {"prediction": prediction[0]}


In [42]:
!uvicorn app:app --reload

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [45492] using StatReload
ERROR:    Error loading ASGI app. Could not import module "app".
INFO:     Stopping reloader process [45492]
^C


## Deployment Thinking

Based on the models  trained, here are some considerations for deployment on different devices:

### Lightweight models (Random Forest / Logistic Regression)

*   **Pros**:
    *   Smaller memory footprint.
    *   Faster inference time.
    *   Lower computational requirements, making them suitable for resource-constrained devices like smartwatches.
*   **Cons**:
    *   May have slightly lower accuracy compared to more complex models, especially on complex datasets.
*   **Deployment Scenario**:
    *   **Smartwatch Edge Inference**: These models are ideal for running directly on smartwatches to provide real-time predictions or insights without needing constant connectivity to a phone or the cloud. This is beneficial for features requiring immediate feedback, like activity tracking analysis or basic health anomaly detection.

### Heavy models (XGBoost / Neural Network)

*   **Pros**:
    *   Generally higher accuracy on complex tasks.
    *   Can capture more intricate patterns in the data.
*   **Cons**:
    *   Larger memory footprint.
    *   Higher computational requirements, which can drain battery and require more processing power.
    *   Slower inference time compared to lightweight models.
*   **Deployment Scenario**:
    *   **Cloud Sync (Phone + Watch Integration)**: These models are better suited for deployment in the cloud or on a connected smartphone where more computational resources are available. The smartwatch can collect data and sync it to the phone/cloud for more in-depth analysis and model inference. This is useful for tasks like long-term health trend analysis, personalized recommendations, or detecting more subtle patterns that require a powerful model.

### Hybrid Approach

A common and effective approach is to use a hybrid model:

*   Deploy a lightweight model on the smartwatch for real-time, basic inference.
*   Sync data to the cloud/phone periodically to run a heavier model for more complex analysis and to refine the lightweight model's predictions or update it.

This leverages the strengths of both types of models, providing both immediate feedback on the device and more sophisticated analysis when connected.